#  제8장

In [1]:
%matplotlib inline
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd
import os
from ipypublish import nb_setup

In [2]:
from statsmodels.compat import lzip
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [3]:
df = pd.read_csv("education2020.csv")

In [4]:
df1 = df.loc[df['number'].isin([1]) & (df['age'] >= 30) & (df['age'] < 39)] # 30대 1인가구
df2 = df1.copy()
edu_model = ols("income ~ education_year", data=df2).fit()
print(edu_model.summary())

                            OLS Regression Results                            
Dep. Variable:                 income   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     14.92
Date:                Mon, 13 Feb 2023   Prob (F-statistic):           0.000133
Time:                        15:55:49   Log-Likelihood:                -3203.2
No. Observations:                 352   AIC:                             6410.
Df Residuals:                     350   BIC:                             6418.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        408.7517    826.622      0.

In [5]:
df3 = df2.copy()

In [6]:
df3 = df3[['education_year', 'income']]

In [7]:
df3.head()

,education_year,income
19,14,1089
134,16,3756
157,16,2004
190,16,5802
191,16,3032


## RMSE계산

### 1. $\hat{\sigma}$을 구한다.

In [8]:
intercept =edu_model.params[0]
slope = edu_model.params[1]
df3['fitted'] = intercept + df3['education_year'] * slope
df3['sse'] = (df3['income'] - df3['fitted'])**2
sse = df3[['sse']]
n = len(df3.index)
k = 2
sigma_hat = np.sqrt(np.sum(sse)/(n-k))
np.round(sigma_hat, 2)

sse    2172.88
dtype: float64

### 2. $SE(a)$을 구한다.

In [10]:
df3['x_bar'] = df3['education_year'].mean()
x_bar = df3['education_year'].mean()
df3['x_residual2'] = (df3['education_year'] - df3['x_bar'])**2
x_residual2 = np.sum(df3['x_residual2'])
n = len(df3.index)
se_a = np.sqrt(1/n + x_bar**2/x_residual2) * sigma_hat

In [11]:
np.round(se_a, 2)

sse    826.62
dtype: float64

### 2. $SE(b)$을 구한다.

In [12]:
se_b = sigma_hat/np.sqrt(x_residual2)

In [13]:
se_b

sse    55.542892
dtype: float64

### 3. $a,b$에 대한 신뢰구간을 구한다.

귀무가설 $H_0 : \beta = \beta_0(\beta_0$는 주어진 상수)라는 귀무가설을 검정한다고 하자

만약 귀무가설 $H_0 : \beta = 0$ 즉 x가 y를 설명하는데 무용지물이라는 귀무가설을 5% 유의수준에서 검정한다고 하자.    
95%에 대한 신뢰구간은 $b\pm2SE(b)$가 된다. 해당 구간에 0$(=\beta_0)$을 포함하고 있는지 살펴본다.     
해당 구간에 0을 포함하지 않으면 귀무가설을 기각한다.

In [14]:
# a에 대한 신뢰구간
lb_a = intercept - 2 * se_a
hb_a = intercept + 2 * se_a

In [15]:
np.round(lb_a,2)

sse   -1244.49
dtype: float64

In [16]:
np.round(hb_a,2)

sse    2062.0
dtype: float64

In [17]:
# b에 대한 신뢰구간
lb_b =  slope  - 2 * se_b
hb_b =  slope + 2 * se_b

In [18]:
np.round(lb_b,2)

sse    103.49
dtype: float64

In [19]:
np.round(hb_b, 2)

sse    325.66
dtype: float64

### 4. $a,b$에 대한 t값을 구한다.

$ \Large t = \frac{b - \beta_0}{SE(b)}$

In [20]:
t_a = intercept/se_a

In [21]:
np.round(t_a, 3)

sse    0.494
dtype: float64

In [22]:
t_b = slope/se_b

In [23]:
np.round(t_b, 3)

sse    3.863
dtype: float64

### 5. $a,b$에 대한 p값을 구한다.

In [24]:
from scipy import stats
df = len(df3.index)-2 
p_a = (1 - stats.t.cdf(t_a,df=df))*2
p_b = (1 - stats.t.cdf(t_b,df=df))*2

In [25]:
np.round(p_a, 4)

array([0.6213])

In [26]:
np.round(p_b, 4)

array([0.0001])